```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Figure maker for comparing schemes

This notebook is for creating informative figures for comparing different oceanographic properties between numerical schemes.
All data is expected to be read in from NetCDF files.



#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec

import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(5.0, 4.0))
rc('animation', html='html5')

#Import our Simulation Reader
from SWESimulators import SimReader


## Linear Coriolis

In this case, slow developing effects caused by non-constant Coriolis forces are considered.
Let the Coriolis parameter be defined as 
$$f(y) = \tilde{f} + \beta y,$$
on a region with flat bottom topography.
Consider then an initial state similar to in the case of Rossby adjustment (ref to equation), and let the boundary be open, so that initial energy is allowed to leave the domain.
Instead of the steady state obtained in Section XX, the varying Coriolis will produce slowgoing NAME OF WAVES waves travelling westwards.

Capturing slow-developing effects by numerical methods are sometimes challenging, as derivatives have a tendency to be smoothed by certain schemes. 
This case will therefore test how well the selected numerical schemes can handle such waves.

Since we here are interested in long-term development, we look at the overall result at the latest time step.

In [ ]:
# Define file names
folder = "netcdf_2017_12_12/"
fbl_file_name =   folder+"FBL_2017_12_12-09_54_34.nc"
ctcs_file_name =  folder+"CTCS_2017_12_12-09_56_14.nc"
kp07_file_name =  folder+"KP07_2017_12_12-09_57_27.nc"
cdklm_file_name = folder+"CDKLM16_2017_12_12-09_57_12.nc"
cdklm_double_file_name = "netcdf_2017_12_18/CDKLM16_2017_12_18-13_06_43.nc"

In [ ]:
reload(SimReader)
fbl_sim = SimReader.SimNetCDFReader(fbl_file_name)
ctcs_sim = SimReader.SimNetCDFReader(ctcs_file_name)
kp07_sim = SimReader.SimNetCDFReader(kp07_file_name)
cdklm_sim = SimReader.SimNetCDFReader(cdklm_file_name)
cdklm_double_sim = SimReader.SimNetCDFReader(cdklm_double_file_name)

def plotLinearCoriolis(sim, handnoteta=False):
    eta, hu, hv, t = sim.getLastTimeStep()
    if handnoteta:
        eta = eta - 60
    fig = plt.figure(figsize=(8, 8)) 
    gs = gridspec.GridSpec(2, 1, height_ratios=[2.5, 1]) 
    ax0 = plt.subplot(gs[0])
    ax0imshow = ax0.imshow(eta)
    fig.colorbar(ax0imshow, ax=ax0)
    plt.title(sim.get('simulator_short') + " at t = " + str(t))
    ax0imshow.set_clim(-0.02, 0.02)
    
    ax1 = plt.subplot(gs[1])
    sim.makeInfoPlot(ax1)
    
    fig.tight_layout()
    
plotLinearCoriolis(fbl_sim)
plotLinearCoriolis(ctcs_sim)
plotLinearCoriolis(kp07_sim, True)
plotLinearCoriolis(cdklm_sim, True)
plotLinearCoriolis(cdklm_double_sim)


#### Collection of results for linear Coriolis

In [ ]:
reload(SimReader)
folder = "netcdf_2017_12_14/"
fbl_file_name =       folder+"FBL_2017_12_14-15_11_03.nc"
ctcs_file_name =     folder+"CTCS_2017_12_14-15_12_56.nc"
kp07_file_name =     folder+"KP07_2017_12_14-15_15_22.nc"
cdklm_file_name = folder+"CDKLM16_2017_12_14-15_14_05.nc"

sims = [SimReader.SimNetCDFReader(fbl_file_name), \
        SimReader.SimNetCDFReader(ctcs_file_name), \
        SimReader.SimNetCDFReader(kp07_file_name), \
        SimReader.SimNetCDFReader(cdklm_file_name) ]

for i in range(0, sims[0].getNumTimeSteps(),2):
    fig = plt.figure(figsize=(10,10))
    t = 0
    for sim_no in range(len(sims)):
        eta, hu, hv, t = sims[sim_no].getTimeStep(i)
        ax0 = plt.subplot(2,2,sim_no+1)
        ax0imshow = ax0.imshow(eta)
        ax0imshow.set_clim(-0.06, 0.06)
        fig.colorbar(ax0imshow, ax=ax0)
        plt.title(sims[sim_no].get('simulator_short') + " at t = " + str(t))
    fig.savefig("linearCoriolis_comparison_{:010.0f}.png".format(t))

### Long term effect of the same Linear Coriolis case:
Plotten in the $x$-$t$ plane at three different $y$-values.

In [ ]:
reload(SimReader)
folder = "netcdf_2017_12_18/"
fbl_file_name =       folder+"FBL_2017_12_18-13_00_40.nc"
ctcs_file_name =     folder+"CTCS_2017_12_18-13_04_05.nc"
kp07_file_name =     folder+"KP07_2017_12_18-13_10_39.nc"
cdklm_file_name = folder+"CDKLM16_2017_12_18-13_06_43.nc"


sims = [SimReader.SimNetCDFReader(fbl_file_name), \
        SimReader.SimNetCDFReader(ctcs_file_name), \
        SimReader.SimNetCDFReader(kp07_file_name), \
        SimReader.SimNetCDFReader(cdklm_file_name) ]

for sim in range(4):
    fig = plt.figure(figsize=(15,10))
    T = sims[sim].getNumTimeSteps()
    nx = sims[sim].get('nx')
    xtplaneMid = np.zeros((T ,nx),  dtype=np.float32, order='C')
    xtplaneLower = np.zeros((T ,nx),  dtype=np.float32, order='C')
    xtplaneUpper = np.zeros((T ,nx),  dtype=np.float32, order='C')
    mid_y  = sims[sim].get('ny')/ 2
    low_y  = mid_y - 10
    high_y = mid_y + 10
    for t in range(T):
        xtplaneMid[t, :] = sims[sim].getEtaXSlice(t, mid_y)
        xtplaneLower[t, :] = sims[sim].getEtaXSlice(t, low_y)
        xtplaneUpper[t, :] = sims[sim].getEtaXSlice(t, high_y)
    
    #gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    #ax0 = plt.subplot(gs[0])
    #ax0imshow = ax0.imshow(xtplane, origin="lower", )
    #ax0imshow.set_clim(-0.06, 0.12)
    #fig.colorbar(ax0imshow, ax=ax0)
    #ax1 = plt.subplot(gs[1])
    #sims[sim].makeInfoPlot(ax1)
    domain_extent = [ 0, sims[sim].get('dx')*nx, 0, sims[sim].ncfile.variables['time'][-1] ]

    gridspec.GridSpec(3, 3)
    ax0 = plt.subplot2grid((3,3), (0,0), rowspan=2)
    ax0imshow = plt.imshow(xtplaneLower, origin="lower", extent=domain_extent)
    ax0imshow.set_clim(-0.06, 0.12)
    fig.colorbar(ax0imshow, ax=ax0)
    plt.title('y = (ny/2 - 10)*dy')
    plt.xlabel('x')
    plt.ylabel('Time')
    plt.axis('tight')
    
    ax0 = plt.subplot2grid((3,3), (0,1), rowspan=2)
    ax0imshow = plt.imshow(xtplaneMid, origin="lower", extent=domain_extent)
    ax0imshow.set_clim(-0.06, 0.12)
    fig.colorbar(ax0imshow, ax=ax0)
    plt.title('y = (ny/2)*dy')
    plt.xlabel('x')
    plt.ylabel('Time')
    plt.axis('tight')

    ax0 = plt.subplot2grid((3,3), (0,2), rowspan=2)
    ax0imshow = plt.imshow(xtplaneUpper, origin="lower", extent=domain_extent)
    ax0imshow.set_clim(-0.06, 0.12)
    fig.colorbar(ax0imshow, ax=ax0)
    plt.title('y = (ny/2 + 10)*dy)')
    plt.xlabel('x')
    plt.ylabel('Time')
    plt.axis('tight')

    ax1 = plt.subplot2grid((3,3), (2,0), rowspan=1, colspan=3)    
    sims[sim].makeInfoPlot(ax1)
    
    simName = sims[sim].get('simulator_short')
    fig.suptitle("Eta for " + simName, fontsize=22)
    fig.savefig("linearCoriolis_beta_plane_" + simName + ".png".format(t))
        
# Compare last timestep between them all:
def lastTimeStepImShow(sim, ax0):
    eta, hu, hv, t = sim.getLastTimeStep()
    
    ax0imshow = ax0.imshow(eta)
    fig.colorbar(ax0imshow, ax=ax0)
    plt.title(sim.get('simulator_short') + " at t = " + str(t))
    ax0imshow.set_clim(-0.02, 0.02)
   

fig = plt.figure(figsize=(8, 8)) 
gs = gridspec.GridSpec(2, 2)
for i in range(4):
    ax0 = plt.subplot(gs[i])
    lastTimeStepImShow(sims[i], ax0)
fig.suptitle("Eta for at last timestep", fontsize=14)
fig.savefig("linearCoriolis_last_timestep.png".format(t))
#fig.tight_layout()

## Simple test of functionality in SimReader

In [ ]:
reload(SimReader)
fbl_sim = SimReader.SimNetCDFReader(fbl_file_name, ignore_ghostcells=True)
ctcs_sim = SimReader.SimNetCDFReader(ctcs_file_name)
kp07_sim = SimReader.SimNetCDFReader(kp07_file_name)
cdklm_sim = SimReader.SimNetCDFReader(cdklm_file_name)
print "--------"
fbl_sim.printVariables()
print "--------"
fbl_sim.printAttributes()
print fbl_sim.getNumTimeSteps()
eta, hu, hv, t = fbl_sim.getLastTimeStep()
print t
fig1 = plt.figure()
plt.imshow(eta)
plt.colorbar()

print eta.shape
print hu.shape
print hv.shape

subfig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,8))
cdklm_eta, cdklm_hu, cdklm_hv, cdklm_t = cdklm_sim.getLastTimeStep()
ax1Im = ax1.imshow(cdklm_eta)
subfig.colorbar(ax1Im, ax=ax1)
cdklm_sim.makeInfoPlot(ax2)

fig = plt.figure(figsize=(8, 8)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
ax0 = plt.subplot(gs[0])
ax0imshow = ax0.imshow(cdklm_eta)
fig.colorbar(ax0imshow, ax=ax0)
ax1 = plt.subplot(gs[1])
cdklm_sim.makeInfoPlot(ax1)

plt.tight_layout()
#print cdklm_t
#kp07_eta, kp07_hu, kp07_hv, kp07_t = kp07_sim.getLastTimeStep()
#ax2im = ax2.imshow(kp07_eta)
#fig2.colorbar(ax2im, ax=ax2)

#ctcs_sim.makeAnimation()